In [ ]:
import os, sys

print('Importing dataset...')

from google.colab import drive
drive.mount('/content/drive')
os.chdir("drive/My Drive/DPR_QANTA")

Importing dataset...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget "https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.train.2018.04.18.json"

--2021-05-19 01:06:40--  https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.train.2018.04.18.json
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.237.248
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.237.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142113194 (136M) [application/json]
Saving to: ‘qanta.train.2018.04.18.json’

qanta.train.2018.04 100%[===================>] 135.53M  31.9MB/s    in 5.0s    

2021-05-19 01:06:46 (27.1 MB/s) - ‘qanta.train.2018.04.18.json’ saved [142113194/142113194]



In [ ]:
!wget "https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.dev.2018.04.18.json"

--2021-05-19 02:37:17--  https://s3-us-west-2.amazonaws.com/pinafore-us-west-2/qanta-jmlr-datasets/qanta.dev.2018.04.18.json
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.248.40
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.248.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3090653 (2.9M) [application/json]
Saving to: ‘qanta.dev.2018.04.18.json’

qanta.dev.2018.04.1 100%[===================>]   2.95M  2.62MB/s    in 1.1s    

2021-05-19 02:37:19 (2.62 MB/s) - ‘qanta.dev.2018.04.18.json’ saved [3090653/3090653]



In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-eo9mfhe8
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-eo9mfhe8
  Created wheel for farm-haystack: filename=farm_haystack-0.8.0-cp37-none-any.whl size=159099 sha256=a0450d9209cb2115a3f46c6bbca9be54d0e7f9c30ffd5f521e32e579ca80be9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-h80xyhtp/wheels/ab/41/a4/4fbf362de283352078ecb6705c08b6525347aaea2eead2a60c
Successfully built farm-haystack


In [ ]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [ ]:
from haystack.document_store.faiss import FAISSDocumentStore

# Document Store
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

05/19/2021 21:17:58 - INFO - faiss.loader -   Loading faiss with AVX2 support.
05/19/2021 21:17:58 - INFO - faiss.loader -   Loading faiss.
05/19/2021 21:17:59 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
import json

with open('qanta.dev.2018.04.18.json') as f:
  data = json.load(f)

In [ ]:
dict(list(data.items())[:1])

{'questions': [{'answer': 'annexation of the Republic of Texas by the United States [or Annexation Treaty of 1845; accept any answer indicating that Texas is joining the United States of America as a state; do not accept any answer indicating the independence of Texas from Mexico]',
   'category': 'History',
   'dataset': 'protobowl',
   'difficulty': 'College',
   'first_sentence': 'Robert Walker argued that failing to take this action would lead to an overflow of Northern insane asylums and British intervention.',
   'fold': 'guessdev',
   'gameplay': True,
   'page': 'Texas_annexation',
   'proto_id': '55414836ea23cc9417e9ba80',
   'qanta_id': 93135,
   'qdb_id': None,
   'subcategory': 'American',
   'text': "Robert Walker argued that failing to take this action would lead to an overflow of Northern insane asylums and British intervention. Juan Almonte resigned his diplomatic post in indignation over this event. Isaac Van Zandt began discussing this plan with Abel Upshur before Ups

In [ ]:
doc_dir="question_data/wiki"

# Convert files to dicts
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=False)

05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Apology (Plato).txt
05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Battle of Salamis.txt
05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Charles Stewart Parnell.txt
05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Francisco I. Madero.txt
05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Bartleby, the Scrivener.txt
05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Osiris.txt
05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Samuel Beckett.txt
05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Meiji Restoration.txt
05/19/2021 21:20:10 - INFO - haystack.preprocessor.utils -   Converting question_data/wiki/Anne Br

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/drive/My Drive/DPR_QANTA/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.23MB [00:00, 55.2MB/s]


In [ ]:
import sqlite3
sqlite3.sqlite_version  # 3.34.1

'3.34.1'

In [ ]:
!sudo add-apt-repository -y ppa:dqlite/stable
!sudo apt update
!sudo apt-get install -y sqlite3

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.5 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://deve

In [ ]:
# documents to DB
document_store.write_documents(dicts)

05/19/2021 21:48:44 - ERROR - haystack.document_store.sql -   Transaction rollback: UNIQUE constraint failed: document.id


IntegrityError: ignored

## Initalize Retriever, Reader, and Finder

In [ ]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)

# update the document_store
document_store.update_embeddings(retriever)

In [ ]:
 reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

## Pipeline

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)